In [ ]:
#
# 生成训练数据(GenTrainingData)：
#

# -*- coding:utf-8 -*-
import os
import tensorflow as tf
import numpy as np
from PIL import Image
import shutil
import pandas as pd
import cv2
from math import *

# 生成字典文件: 或者用ord(), chr()函数进行整数和中文转换，缺点是数字太大，不连续!
def CreateDict(dictList, dictFileName):
    # fp = open(dictFileName, 'w', encoding='utf-8')
    fp = open(dictFileName, "w+")
    for x, word in enumerate(dictList):
        if dictList[x] != '\n':
            fp.write(dictList[x])
            fp.write("\n")
    return

# 找到单个文字对应的编码：
def GetCharCode(dictList, char):
    x = 0

    if len(char) == 0:
        print("Error: char is none ",char)
        return None

    # 当dictList为空时：避免enumerate报错！
    if len(dictList) == 0:
        dictList.append(char)
        return 0

    for x, ch in enumerate(dictList):
        if dictList[x] == char:
            return x

    # 没有找到，则添加到末尾：
    dictList.append(char)
    # print("Not Found: [", char, "]")
    return x+1

#
# 添加图片文件名和内容到标签文件
# destdir\label.txt: item.filename item.content
#
def InsertLabelItem(dictList, item, destDir, lableFile):
    # print(filename, content)
    oneitem = item[0] + ".jpg"       # item.filename
    # full_lablefile = os.path.join(destDir, lableFile)
    fp = open(lableFile, "a+")
    
    for ch in enumerate(item[1]):       # item.content
        code = GetCharCode(dictList, ch[1])       # 得到对应的编号ch[1]
        oneitem = oneitem + " " + str(code)
        
    oneitem = oneitem + "\n" 
    fp.write(oneitem)
    fp.close()
    
#
# 将图片文件转换TFRecord格式：
#
#def Data2TFRecord(images, labels, filename):
def Data2TFRecord(picPath, lableName, recName):

    if os.path.isfile(recName):
        print("%s exists, now deleted!" % recName)
        os.remove(recName)

    labels = []
    images = []

    #for i,file_path in enumerate(file_path_list):
    train_txt = open(lableName)

    for idx in train_txt:
        idx = idx.rstrip("\n")
        spt = idx.split(' ')
        images.append(os.path.join(picPath, spt[0]))
        labels.append(spt[1:])
    
    #print("FILES: ", images[0:10])
    #print("LABEL[0]: ", labels[0:10])
    print("Converting data into %s ..." % recName)
    cwd = os.getcwd()

    writer = tf.python_io.TFRecordWriter(recName)

    for index, img_name in enumerate(images):
#         img = Image.open(img_name)
#         img = img.convert('L')

#         # 调整大小为统一尺寸：
#         img = img.resize((280, 32))
#         img_raw = img.tobytes()
        
##################分割线   如果宽度小于高度，则旋转90度，不能使用PIL的rotate函数（会导致信息缺失）
        img = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)
        height,width = img.shape[:2]
        if(width < height):
            degree = 90
            heightNew = int(width*fabs(sin(radians(degree)))+height*fabs(cos(radians(degree))))
            widthNew = int(height*fabs(sin(radians(degree)))+width*fabs(cos(radians(degree))))
            matRotation=cv2.getRotationMatrix2D((width/2,height/2),degree,1)
            matRotation[0,2]+=(widthNew-width)/2
            matRotation[1,2]+=(heightNew-height)/2
            img=cv2.warpAffine(img,matRotation,(widthNew,heightNew),borderValue=(255,255,255))
        cv2.imwrite('./tmp.jpg', img, params=None)
        # plt.imshow(img,cmap='gray')

        img2 = Image.open('./tmp.jpg').convert('RGB')#直接转L模式有错？
        img2 = img2.convert('L')
        img2 = img2.resize((280, 32))
        img_raw = img2.tobytes()
##################分割线 
        label = np.asarray(labels[index], dtype=np.int64)
        print(label)
        
        if len(label) < 30: 
            example = tf.train.Example(features=tf.train.Features(feature={
                    'label': tf.train.Feature(int64_list=tf.train.Int64List(value=label)),
                    'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),      # [img_raw]表示为[280,32,1],只考虑1个通道
                }))
        
            writer.write(example.SerializeToString())  # Serialize To String
        
    writer.close()
    
    print("Convert finished!")
    return

# 读取目录及子目录下所有文件：拷贝到指定文件夹

def GenTrainingData(dictList, picDir, destDir, lableFile, recName):
    # 删除目标目录下所有文件：
    if os.path.isfile(recName):
        print("%s exists, now deleted!" % recName)
        os.remove(recName)

    if os.path.isfile(lableFile):
        print("%s exists, now deleted!" % lableFile)
        os.remove(lableFile)
        

    #dataset = pd.read_csv("./data/problem3/content_train.csv", nrows=50)
    dataset = pd.read_csv("./data/problem3/content_train.csv")
    
    # 总行数：
    num = int(dataset.size/2)
    print("Total rows: ", num)
    
    filenames = dataset.iloc[ : , :].values
    
    for i in range(num):
        #print(filenames[i][0], filenames[i][1])
        InsertLabelItem(dictList, [filenames[i][0], filenames[i][1]], destDir, lableFile)
    
    # 生成代码表：
    CreateDict(dictList, './training_data/ocr_char.txt')
    
    # 生成TFRecord：
    Data2TFRecord('./data/problem3/train',lableFile, recName)

    return
dictList = []
GenTrainingData(dictList, './data/problem3/train/', './training_data','./training_data/label_convert.txt', './training_data/ocr_convert.tfrecords')

Total rows:  26439
Converting data into ./training_data/ocr_convert.tfrecords ...
[0 1]
[2 3 4 5 6 7]
[8 9]
[10 11 12 13 14]
[15  0 16 17 18 19]
[20 21 22 23 24 25]
[26 27 28 29 30 31 32 33 34]
[35 36 37 38]
[39 40 41 42]
[43  0 44 45]
[46 47 48]
[24 49 50 51 52 53]
[54 55 56 57 58 59 60 61 62 63 64 65 66 67]
[68 69 70]
[71 72]
[73 74 75 75 76 77 78 79 80 77 76 79 74 81]
[82 83 84]
[85 86]
[87 88]
[89 90]
[91 92 93 94]
[95 73 72 78 74 77 75 96 97 98]
[ 99 100]
[101  55 102 103 104 105 106]
[107 108 109 110 111 112 113 114 112 115 116  64  65  66  67]
[117 118 119 120  39 121 122]
[123 124]
[125 126  17 127]
[ 85 128 128 128 129 129 129   8]
[  8  86   8 128 130 131 131 129 129 129   9]
[132 133 134  34 135  30 136 133 137 138 139]
[140 141 142 143]
[  8  86  85  85  86 144  85 129 131 130   9]
[  0 145]
[146  81  79  79]
[ 85 147]
[ 66  35 148 149]
[ 85   9   9   9 150 130   9   9   9 151]
[131   9 152]
[153 154  11  62 155  42]
[144   9]
[108 111 156 157]
[ 44 158]
[159 160 109 161]
[

[728 289 167 168 450 240 729 730]
[ 98 196 195 190 282 189 475 190 281]
[731 732 346 101 111 683 733 609 295 734 735 247 736 737 738]
[439 614 507]
[739 740]
[741 498 742]
[  9 128]
[ 15 239 321 290 177]
[743 646  44 155 744]
[550  62 177 745]
[746]
[747 482 748 749  37 750]
[ 17 412 203  93 240]
[130 128   9]
[ 73 280 357 566  71 566]
[379 751 752]
[121 753 754]
[755 290]
[  8 131 130  86 129   9  86   9 131 131 130]
[550 129   8 359 317 491 130 129 359 317 756 258 359 317]
[155]
[711 475 476 189  77  81 282]
[321 116 757 477]
[642 379 758]
[ 70 640 257 759 412 118 760  59 761 321 165 295  11 762 378 295 763 215
 467 744 258 764  95  63 215  70 531]
[  8  86 258 129  85   8 129 129 129 129]
[ 74  74 765 709]
[181 182 346  99]
[666 667]
[91 92]
[166 766 767   5 768 166 612 769 156 770]
[131   4]
[144 130  85   9   9   9   9]
[  8 130  85 144 129   8 129 258 131 129 130]
[269 771]
[345 184 125  65 274 529 119 772 773]
[774 421]
[533 380]
[775 340 257 121 776 305  84 520 777 377 520 300 

[367 368 369 370]
[193 194 281 549]
[1040  576  123  470  790 1041  240]
[122 655 643 901]
[1042 1043  884  378 1044]
[ 439 1045  956  319  217]
[553 168 572 776]
[1002 1046]
[ 450  120  502  109 1047  987 1048 1049  741  579]
[ 249  794 1050  241]
[35 36 37 38]
[ 141 1051  181    4  257  484  475  361  192  193   77  194  192  549
  195  194  195  190  193  189  193]
[ 589  433   27  556 1052 1012  487  218  382]
[ 97 317 475 196 188 549 194 281 190 195  77 147 475 359 188 281 196 476
 193 194 281 549]
[ 451  325  379  257 1053  710]
[484 189 282 195 475]
[ 123  470 1054 1055]
[1012  493 1056  564]
[538  17  39 560 443 645  14]
[ 49 455 569]
[ 756 1057]
[350  96  77 484  96  81  79]
[1012]
[ 101  216  204  886  773 1058   48  270  200 1059   36  204  379  246
  184 1014  197  231]
[  8 130   9   8 130 129 128 130 144  85   9]
[ 24  49  93 354]
[ 147  196  281 1060  195  190]
[484  72  97  73 219  72  76 219  74  78 150 173  96  96 350]
[711 219 220 220]
[ 859  363  241 1061 1062 1063]

[ 107  108 1227 1228]
[ 73 711 357]
[ 131  151  913 1229]
[1230  500]
[ 765 1092  839  503]
[ 653   30 1231 1232 1233 1234  479  385  240  311   40]
[ 160   66   67  165  830 1235  618   30   43  381  175  384  272  790
  820  379  169]
[1236 1237 1238]
[ 24  49 790 275 692 608]
[346 619 234 518  83 379 315 349 230]
[1239  936  496]
[  98  194  195  196  195   77  189  193   77  475   77 1070  475  476
   77  281  192  188]
[367 368 369 370]
[131]
[  62 1240  707  290]
[146  81  79  79]
[711 196 189 282 191 195]
[521  93 354 109  70 315 602 379]
[ 24  49  93 354]
[ 93 354  49 455 439 291]
[1112  412  111  421]
[193 317 189 361 195 193]
[1241 1242  206  727]
[159 313 330]
[1243  948  797 1244]
[ 452 1245   57]
[ 159 1246  107]
[1247  444]
[ 110  153 1248  380]
[459 460]
[549 361 475 476 193 189 190 191]
[  8  85 498  85 144 499]
[1223  535 1249 1250  683 1251  861]
[487  33 868]
[1252 1253 1254]
[895]
[201]
[313 377 497 204]
[1255  137  858]
[258   9]
[ 373  301  204  957  577    5  336

[450 109 102]
[ 702 1147   99  100]
[1372 1373  376  569]
[391 128]
[ 96  81  72  79  78  98  77  81  72  97  72  78 146  77  97 219  78  77
  98  79 220  79  95  96 484]
[637  25]
[  39  805  573 1374  210  474]
[250 776 376 552  52  53]
[427 148 516]
[223   5 634]
[71 72 76]
[496 488 562 928]
[1375  330    8 1376]
[ 24  49 957 168 692 608]
[ 85   9   8 131]
[558  28]
[739 740 428 429]
[518 559]
[101 216 437 273 378 612 916  24  25]
[  9 129  86 128  86 131]
[ 118 1377  517  569]
[  8 130 131  85 258  86  86   8   9   8   9]
[   5 1378  611  433]
[722]
[255   4 106]
[ 885 1046  692  608  319  481  168  708  709]
[495 496 497 204]
[   5  691 1223  897]
[1379 1380  257  261  489]
[1381  556 1382 1383   30 1384 1385  505  865]
[ 313 1121]
[ 274 1331 1131]
[980 802 439 291]
[ 159   49 1265  168]
[102  60 408  63 450]
[989]
[1386  789  217  166 1387  131    9    9  150   86    9    9  151   77
 1388  643   86    9    9  150    8   85    9    9  151]
[790 820]
[463 464 295]
[ 382 1389  184 

...

[  92   93   94  900 1607  105  634]
[97 79 95 81 79 98]
[ 85   9 484]
[ 181  182 1066  379  756    3]
[217 663]
[146  96 350  72 351  74]
[1021  442]
[ 97 357 350]
[585 683 327]
[282   9  86 475   9 128 130  85]
[ 720  111 1946]
[ 251  378  584 2756]
[ 35 376]
[2543 2982]
[81]
[ 98  97 146]
[ 411  412   19 1303   56   57   64   65   66   67]
[76 72 97 73 72 96 78]
[ 36 507 988]
[173  72 146  73  98]
[128]
[509 729]
[41]
[  8 130   8 128 130  85 258   8   8 130 130]
[ 123  470  463 1936  529    8  130  129]
[1815  231  741  357   98  351 1017  215   42]
[ 200 1059   36  204  379  246  184 1014  197  231  316]
[ 667 1164 1111  536]
[  49  455 2238 1343  940   56  167  168  453]
[ 97 219  77 566  72  77  76  74  78  77 173  74  77 173  72  74  78]
[319   2]
[492 130 128]
[ 101  216  159 1331  231  270  726  231  710  463  257 1087  570]
[414  38]
[ 241 1253]
[550  62 789 168 217 238]
[240 550]
[553 168  82 664]
[ 150  150 1011  791 1111  239  608]
[ 125 1077  848  497  534 1625 2816]
[  

[790 607 227 804  24  25]
[ 657 1279]
[109 899]
[ 101  216   30 1272  109  396  241 1428]
[ 74 196 193 195 190 475 361]
[ 79 391  72  98]
[74]
[1134  694]
[956 102 253]
[85  4]
[718 987 467]
[470]
[ 536 1453  174  275]
[ 376 1431 1014 1605  252  502   35   36]
[ 241   31  638  294 1090  111  174  275]
[108 548 111]
[1639 2478 1055  841]
[2258 1067]
[ 101  216  204  101   55  295  714  204   23  213 1466 1281  504]
[736]
[ 80 219  95  73  74  78 146 701 219  77  97  73  96 219  76  72 220 219
 566  72  79  77  95  96 484 484 219  78  72  98  76  77  73  79  74 220
  98  73  77  97  79  81 351  72  95  79  77  95  79  78  98  79  81]
[1554 2502 2503   30  339  166 1447 3024]
[1276 1277    6  213  106]
[176  66 482 302]
[ 86 129   9   9  77  85   8 131   8]
[96 95 96 95 96]
[  8 128 258]
[   8  131  151   85 1123]
[101 216 408  50 429 101  55 988]
[ 95 475 717 195]
[319 239  15 199 311]
[ 672 3025  461  482]
[121 460 536 148 714]
[ 134   28 1661 1209  315  379 1302]
[1489 1490]
[1366  186

[176 177  40 274   8  86 128]
[ 85 144]
[1097  518  817 1537  775  179]
[ 95  73  72  78  79  97  79  77  97  98  96  78  79  77 484 219  97  79
 219 484  77  96 173  77  80 219  73  74  78]
[216 940 432 773 338 482]
[1671  433    2    3  278  453]
[346]
[767   5 770  57]
[ 40 272 104]
[   5   33   99   77  231  125 1164   51   77 2243   99   40  204]
[24]
[2073   21  516]
[711  74  78 357  77  96 173  77  95  73  72  78  74]
[  8 215  42]
[  27  168 1648 2036]
[305 308 601]
[141]
[  24  808  379  246 1133  518   49]
[173  79  74  98 219  81  79 350  72  81  96  78  77  97  98  79  74 357]
[ 184  151 1679  375   21  378 1251  378   64  223 2078  316  106  414
  311  887]
[71 98 96]
[ 75  73  72 220  72  75  97]
[921 753  33 482]
[ 155 1124]
[97]
[ 41 813 239]
[741 109]
[ 346  619  234 1331   83  379  315  637  424  349  230   30   39]
[ 86 280 131]
[2433  516]
[1665   67]
[567 350 386  86  98 484 258  86]
[ 756 1057 1072  813  239  744]
[ 95 194 189 190 475  77 484 281 722 189 361 195]

[382]
[531 103 179 932]
[95 73 72 78 74]
[ 97 549 475 189 190  77 193 188 476 361 195]
[492 130  86]
[2228    7  382 1080  457  159   25  473  382]
[339 496 144 258 151 913  77 104]
[484 391 131]
[ 756 1425  396 1425]
[652 330 353]
[195 359 281 361 281 192 196 195]
[ 97  74 220  79]
[ 235 1008  720  507]
[ 73 220]
[266 665]
[317 195 317 281 196 189 195 193]
[1933 1243 1866]
[ 121  585   60  150 1353]
[  24  714 1212  376   35   48  270]
[493 261 421]
[  0  23 213 870 450]
[231 612  70]
[ 639  634 1054  241  530  241]
[101 216 720 107]
[ 73 220  74]
[166  30 637 424 230]
[1974 2043   31   85  173]
[141]
[1313  901 1017  515]
[ 134 1974  414  699  689]
[ 86   9   9 151]
[  8 130 258   9 129   8 128  85 144   8 129]
[ 98  79 220   9  85 129]
[1054 1037  444 1016  320]
[909 240]
[653 654 337]
[45 45]
[  60 1055  333  861   63 1504  745  315 1066  262 1127  320   65  776
  346  257   60   85  503   63]
[351 189 190 282 475]
[130   8 152]
[ 80  74  78 350  74  77  75 220  74  71  74]
[ 17  1

[409 377 744]
[1992  421 1307  887]
[1042  278]
[ 85   8 144 129]
[530 156  56 256 257]
[ 95  96 484 711  72  78  74  98  72  96  78]
[1013  261   44   45]
[1213  482  303  279]
[ 414  240  562 2248   30  241  118]
[  88  262 1852  709]
[1203  106]
[ 97  98  74  98  79  77 146  81  72 350]
[ 86  85 144 131   9   8 131  86]
[  8   9  66 794 384 462]
[ 85 144]
[ 78 219  95 351 351  98]
[502 421]
[181 324 770]
[ 134  315 1111 2200   62  102]
[382  54 981]
[241 109 479 876]
[231 249 990]
[181 182]
[ 669  948 2333  629  291  750]
[948 174 257 278 453  77 105 276 169]
[  70 2996   12 1915 2297   37  311   30  120 3050  409 1899   33 1798
  404]
[1033  166  612]
[350 196 475 191 281 190]
[530]
[ 286  908  824 1181]
[1056 1607]
[1895  714]
[2068 2749 1230   30   18  424  818   30  255 2164  863  844  396  576
 1913  404]
[ 24  49 389]
[  58 1834  692 1970   58  146  189  190  717  191  281  475  359  195
  475  195]
[208   5]
[137 858 918 477 727 943   4 602 944]
[167 168   2 903 257  86 257  